In [18]:

import pandas as pd
import LinkedInInfo 
import spacy
from LinkedInInfo import fetch_specific_linkedin_info
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [19]:
import textstat

# Example usage
profile_desired = LinkedInInfo.fetch_linkedin_info()
recepient_school = fetch_specific_linkedin_info(profile_desired)
position_name = fetch_specific_linkedin_info(profile_desired)
company_name = fetch_specific_linkedin_info(profile_desired)

def generate_email(your_name, your_field, your_university, recepient_name, position_name, company_name, email_type):
    
    # case for the type of email
    if email_type == ('general'):
        template = (f'Hi {recepient_name},'
                    f'I hope you are doing well! A quick introduction, my name is {your_name} and I am currently a rising senior at {your_university} studying {your_field}. I wanted to reach out because I am interested in the {your_field} field and to learn more about your role & career path at as a {position_name} at {company_name}.'
                    f'I would love to set up a time to chat to learn more about your experiences at your company. I have attached my resume for your reference. Please let me know if you are available as I am happy to work around your schedule to find a time to speak!'
                    f'Thank gonna you for your time and I look forward to hearing from you!'
                    f''
                    f'Best regards,'
                    f'{your_name}')
    
    # case for the type of email
    elif email_type == ('school'):
        template = (f'Hi {recepient_name},'
                    f'I hope you are doing well! A quick introduction, my name is {your_name} and I am currently a rising senior studying {your_field}. I noticed you are a fellow alumni member of {your_university} so I was very excited to reach out!'
                    f'I wanted to reach out because I am interested in the {your_field} field and to learn more about your role & career path at as a {position_name} at {company_name}.'
                    f'I would love to set up a time to chat to learn more about your experiences at your company. I have attached my resume for your reference. Please let me know if you are available as I am happy to work around your schedule to find a time to speak!'
                    f'Thank you for your time and I look forward to hearing from you!'
                    f''
                    f'Best regards,'
                    f'{your_name}')
    else:
        print('Choose a Valid Email Type')
    
    return template



In [20]:
template = generate_email('Aaroh Jugulum', 'Data Science & Mathematics', 'Northeastern University', 'Dave', 
                          position_name, company_name, 'general')

# def a set of stop words 
stop_words = set(["i", "me", "my", "myself", "we", "our", "ours", "you", "your", "he", "his", "she", "her", "it", "its"])

# clean text manually
def clean_text(text):
    tokens = text.lower().split()
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words] 
    return " ".join(filtered_tokens)

template_clean = clean_text(template)
template

'Hi Dave,I hope you are doing well! A quick introduction, my name is Aaroh Jugulum and I am currently a rising senior at Northeastern University studying Data Science & Mathematics. I wanted to reach out because I am interested in the Data Science & Mathematics field and to learn more about your role & career path at as a Northeastern University at Volunteer.I would love to set up a time to chat to learn more about your experiences at your company. I have attached my resume for your reference. Please let me know if you are available as I am happy to work around your schedule to find a time to speak!Thank gonna you for your time and I look forward to hearing from you!Best regards,Aaroh Jugulum'

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# def a set of stop words 
stop_words = set(["i", "me", "my", "myself", "we", "our", "ours", "you", "your", "he", "his", "she", "her", "it", "its"])

# clean text manually
def clean_text(text):
    tokens = text.lower().split()
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words] 
    return " ".join(filtered_tokens)

template_clean = clean_text(template)
template_clean

'hi hope are doing a quick name is aaroh jugulum and am currently a rising senior at northeastern university studying data science wanted to reach out because am interested in the data science mathematics field and to learn more about role career path at as a northeastern university at would love to set up a time to chat to learn more about experiences at have attached resume for please let know if are available as am happy to work around schedule to find a time to gonna for time and look forward to hearing from jugulum'

In [22]:
import re
# helper functions
def sentance_count(text):
    sentance_list = list(text.split('.'))
    sentance_count = len(sentance_list)
    return sentance_count

def wordy_char_count(text):
    sentences = re.split(r'[.!?]', text) 
    over_count = sum(1 for sentence in sentences if len(sentence.strip()) > 150)
    return over_count

In [23]:
print(textstat.flesch_reading_ease(template))  # Higher score = easier to read
print(textstat.flesch_kincaid_grade(template))  # U.S. school grade level
print(textstat.gunning_fog(template))  # Complexity based on word length & sentences
print(textstat.smog_index(template))  # Measures polysyllabic words (ideal for short texts)
print(textstat.automated_readability_index(template))  # Uses characters per word for grade level
print(textstat.dale_chall_readability_score(template))  # Uses a list of "common" words for difficulty
print(textstat.reading_time(template))  # Estimated reading time in seconds

64.2
8.2
8.78
10.7
8.1
7.84
8.43


In [24]:
nlp = spacy.load("en_core_web_sm")


def analyze_readability(text):
    scores = {
        "Flesch-Kincaid Grade Level": textstat.flesch_kincaid_grade(text),
        "Gunning Fog Index": textstat.gunning_fog(text),
        "Dale-Chall Readability Score": textstat.dale_chall_readability_score(text),
        "Flesch Reading Ease": textstat.flesch_reading_ease(text)
    }
    return scores


def analyze_professionalism(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(text)["compound"] 

    doc = nlp(text)
    complex_words = sum(1 for token in doc if token.is_alpha and len(token.text) > 12) 
    num_sentences = len(list(doc.sents))
    avg_sentence_length = len(text.split()) / num_sentences if num_sentences > 0 else 1
    over_count = wordy_char_count(template) 
    sentance_counts = sentance_count(template)
  
    informal_words = {"gonna", "wanna", "lemme", "gotta", "bro", "dude", "lol", "omg", "lmao", "cuz"
                      'love', 'very'}
    
    overly_formal_words = {"utilize", "commence", "terminate", "ascertain", "disseminate", "facilitate", "execute",
                           "expedite", "engender", "convey"}
    
    hedging_words = {"maybe","probably","possibly","perhaps","seems","appears","might","could","somewhat",
                     "relatively"}


    # count the number of types of instances 
    informal_instances = list(1 for i in text.lower().split() if i in informal_words)
    informal_count = sum(1 for i in text.lower().split() if i in informal_words)
    overly_formal_instances = list(1 for i in text.lower().split() if i in overly_formal_words)
    overly_formal_count = sum(1 for i in text.lower().split() if i in overly_formal_words)
    hedging_instances = list(1 for i in text.lower().split() if i in hedging_words)
    hedging_count = sum(1 for i in text.lower().split() if i in hedging_words)
   
   # calc the scores 
    calc_score = (
        (1 - abs(sentiment_score)) * 40  
        + (1 - (complex_words / max(len(text.split()), 1))) * 20 
        + (1 - informal_count / max(len(text.split()), 1)) * 20  
        + (1 if avg_sentence_length > 8 else 0) * 20 
        + (1 - over_count / sentance_counts) * 20
        + (1- overly_formal_count / max(len(text.split()), 1)) * 20
        + (1- hedging_count / max(len(text.split()), 1)) * 20
    )
    #print((complex_words / max(len(text.split()), 1)))
    #print((informal_count / max(len(text.split()), 1)))
    #print((hedging_count / max(len(text.split()), 1)))
    #print((overly_formal_count / max(len(text.split()), 1)))
    calc_score = min(max(calc_score, 0), 100)
    
    ease_read = textstat.flesch_reading_ease(template)
    polly_syllabic = textstat.smog_index(template)
    time_to_read = textstat.reading_time(template)
    
    # define length scoring 
    if time_to_read > 8:
        calc_score += -2
        print('Takes too long to read this email, recipient will lose interest')
    elif polly_syllabic > 10:
        calc_score += -2
    else:
        calc_score = calc_score
        print('Too many words that are not good for short text')
    
    
    # define the readability scores
    scores = analyze_readability(template)
    vals = list(scores.values())
    if vals[0] > 8:
        calc_score += -2
    elif vals[1] < 7:
        calc_score += -2
    elif vals[2] < 8:
        calc_score += -2
    elif vals[3] < 70:
        calc_score += -2
    else:
        calc_score = calc_score
    
    return calc_score

In [25]:
calc_scores = analyze_professionalism(template)
calc_scores 

Takes too long to read this email, recipient will lose interest


96

In [26]:
def count_error_sentance(template):
    template_list = template.lower().split('.')
    
    # define the list of words 
    informal_words = {"gonna", "wanna", "lemme", "gotta", "bro", "dude", "lol", "omg", "lmao", "cuz"
                          'love', 'very'}
        
    overly_formal_words = {"utilize", "commence", "terminate", "ascertain", "disseminate", "facilitate", "execute",
                               "expedite", "engender", "convey"}
        
    hedging_words = {"maybe","probably","possibly","perhaps","seems","appears","might","could","somewhat",
                         "relatively"}
    
    # init the analyzer object 
    analyzer = SentimentIntensityAnalyzer()
    
    # init the dict 
    scores_dict = dict()
    
    # loop through each sentance and give a score
    for idx, i in enumerate(template_list):
        
        # sentiment score for each sentence 
        sentiment_score = analyzer.polarity_scores(i)["compound"] 
        
        # init a nlp object to perform further analysis in the future 
        doc = nlp(i)
        
        # types of instances 
        informal_instances = list(j for j in i.lower().split() if j in informal_words)
        overly_formal_instances = list(j for j in i.lower().split() if j in overly_formal_words)
        hedging_instances = list(j for j in i.lower().split() if j in hedging_words)
        
        # counts
        informal_count = sum(1 for j in i.lower().split() if j in informal_words) 
        overly_formal_count = sum(1 for j in i.lower().split() if j in overly_formal_words)
        hedging_count = sum(1 for j in i.lower().split() if j in hedging_words)
        
        # sentence score 
        sentence_score = ((1 - abs(sentiment_score)) * 40
        + (1 - informal_count / max(len(i.split()), 1)) * 20  
        + (1- overly_formal_count / max(len(i.split()), 1)) * 20
        + (1- hedging_count / max(len(i.split()), 1)) * 20
        )
       
        # round
        sentence_score = round(sentence_score, 2)
        
        # update the dict
        scores_dict[idx] = (sentence_score, sentiment_score, informal_count, overly_formal_count, hedging_count)
        
    return scores_dict

In [27]:
def instance_sentance(template):
    template_list = template.lower().split('.')
    
    # define the list of words 
    informal_words = {"gonna", "wanna", "lemme", "gotta", "bro", "dude", "lol", "omg", "lmao", "cuz"
                          'love', 'very'}
        
    overly_formal_words = {"utilize", "commence", "terminate", "ascertain", "disseminate", "facilitate", "execute",
                               "expedite", "engender", "convey"}
        
    hedging_words = {"maybe","probably","possibly","perhaps","seems","appears","might","could","somewhat",
                         "relatively"}
    
    # loop through each sentance and give a score
    for idx, i in enumerate(template_list):
        # types of instances and append to the list 
        informal_instances = list(j for j in i.lower().split() if j in informal_words)
        overly_formal_instances = list(j for j in i.lower().split() if j in overly_formal_words)
        hedging_instances = list(j for j in i.lower().split() if j in hedging_words)
    
    
    return informal_instances, overly_formal_instances, hedging_instances


In [28]:
def give_feedback_weak(template):
    sentence_score = count_error_sentance(template)
    
    weak_list_sentences = list()
    for i in sentence_score.keys():
        critique_list = list(sentence_score[i])
        if critique_list[0] < 75:
            weak_list_sentences.append(i)
            
    informal_instance = instance_sentance(template)[0]
    overly_formal_instance = instance_sentance(template)[1]
    hedging_instance = instance_sentance(template)[2]
    
    crit_dict = dict()
    for i in weak_list_sentences:
        critique_list_weak = list(sentence_score[i])
        crit_1 = crit_2 = crit_3 = crit_4 = None
        if critique_list_weak[1] < 0.5:
            crit_1 = (f'You are not enthusiastic enough the audience will be disengaged, try using more reinforcing and '
                  'positive language within the email.')
        elif critique_list_weak[2] > 0:
            crit_2 = (f'You are using some informal language in this sentence. These are the instance(s): {informal_instance}')
        elif critique_list_weak[3] > 0:
            crit_3 = (f'You are too formal, you want to keep the language digestible.These are the instance(s): {overly_formal_instance}')
        elif critique_list_weak[4] > 0:
            crit_4 = (f'You are using words that reduce your credibility, you want to be firm and direct. These are the instance(s): {hedging_instance}')
        
        crit_dict[i] = tuple(j for j in [crit_1, crit_2, crit_3, crit_4] if j != None)
        
        non_empty_crit = {k: v for k, v in crit_dict.items() if v}
        
        # return a message as the output 
        message_list = list()
        for i in non_empty_crit.keys():
            message = f'Sentence {i} has the issue. {non_empty_crit[i][0]}'
            message_list.append(message)
    
    return message_list       

In [29]:
message = give_feedback_weak(template)
message 

["Sentence 4 has the issue. You are using some informal language in this sentence. These are the instance(s): ['gonna']"]